<a href="https://colab.research.google.com/github/Graylab/GeoDock/blob/main/GeoDock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GeoDock**: Fast, Flexible Protein-Protein Docking

Official notebook for [GeoDock](www.biorxiv.org/content/10.1101/2023.06.29.547134v1): Flexible Protein-Protein Docking with a Multi-Track Iterative Transformer.

In [1]:
#@title Hit `Runtime` -> `Run all`, then input docking partner1 and partner2

from google.colab import files

out_name = 'test' #@param {type:"string"}

##@markdown Perform structural refinement with OpenMM
#do_refine = False #@param {type:"boolean"}
# refinement is not yet supported
do_refine = False

upload1 = files.upload()
upload2 = files.upload()
partner1 = list(upload1.keys())[0]
partner2 = list(upload2.keys())[0]

Saving 1AVX_l_b.pdb to 1AVX_l_b.pdb


Saving 1AVX_l_u.pdb to 1AVX_l_u.pdb


In [2]:
#@title Install dependencies (~1 min)
%%time
import sys
import torch

pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install geodock
!pip install torch-geometric
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install py3Dmol

# download model weights
!git clone https://github.com/Graylab/GeoDock.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=e9c79e3e01ebac22b6340a60c53b36afa381ebd9482dcc9ae5dee73c54931747
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch

In [6]:
#@title Run Prediction
from geodock.GeoDockRunner import GeoDockRunner
torch.cuda.empty_cache()
ckpt_file = "/content/GeoDock/geodock/weights/dips_0.3.ckpt"
geodock = GeoDockRunner(ckpt_file=ckpt_file)
pred = geodock.dock(
    partner1=partner1,
    partner2=partner2,
    out_name=out_name,
    do_refine=do_refine,
    use_openmm=True,
)

Completed embedding in 0.30 seconds.
File './test.pdb' deleted successfully.
Completed docking in 1.23 seconds.


In [7]:
#@title Display 3D structure {run: "auto"}
import py3Dmol

color = "chain" #@param ["chain", "lDDT", "rainbow"]

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(out_name+'.pdb','r').read(),'pdb')
if color == 'chain':
  view.setStyle({'chain': 'A'},{'cartoon': {'color': 'green'}})
  view.setStyle({'chain': 'B'},{'cartoon': {'color': 'blue'}})
elif color == 'lDDT':
  view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
elif color == 'rainbow':
  view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
#@title Download results

#@markdown Download the predicted pdb file. If download fails, results are also accessible from file explorer on the left panel of the notebook.

from google.colab import files
files.download(f"{out_name}.pdb")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Instructions <a name="Instructions"></a>
**Quick start**
1. Enter the name of your output.
2. Press "Runtime" -> "Run all".
3. Upload docking partner1 and partner2.

**Known issues**
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the large protein complexes.